In [1]:
# ARGS & PARAMS
# if _user_selected_model is None:
#     Execution_Id = "output/{}/Job_{}_best_{}".format(dataset_name, _id,datetime.datetime.now())
# Pycaret Args
from IPython.display import HTML
from IPython.display import Markdown as md
from pyfiglet import Figlet
import datetime
import yaml
import os
# Read YAML file
with open("auto_config.yml", 'r') as stream:
    config = yaml.safe_load(stream)

for i,j in config.items():
    if j == 'None':
        config[i] = None
               
f = Figlet(font='slant')
print (f.renderText('SimpleML'))

print("Type : Classification")
print("User : {}".format(config['user_name']))
print("Analysis name: {}_{}_{}".format(config['analysis_name'], config['user_selected_model'], datetime.datetime.now().strftime ("%Y%m%d")))
print("Model selection : {}".format(config['user_selected_model']))
start_date = datetime.datetime.now()
print("Date time : {}".format(start_date))



# Create analysis folder

try:
    output_folder = "{}/{}/{}/outputs/{}/".format(config['user_name'], config['analysis_name'], config['user_selected_model'], start_date)
    os.makedirs(output_folder)
except FileExistsError:
    # directory already exists
    pass

# backup config
# from shutil import copyfile
# copyfile(src, dst)
#default dataset target variable mappings
_default_target_variables = {'automobile':'price',
'bike':'cnt',
'boston':'medv',
'concrete':'strength',
'diamond':'Price',
'forest':'area',
'gold':'Gold_T',
'house':'SalePrice',
'insurance':'charges',
'parkinsons','PPE',
'traffic':'traffic_volume'}

# Caret config
if config['demo_run']:
    _demo_dataset = config['demo_dataset']
    _target = _default_target_variables[_demo_dataset]
    _target_class = _default_target_variables[_demo_dataset]
    print("Input data: {}".format(config['demo_dataset']))
else:
    _input_file = config['input_file']
    # Target processing
    _target = config['target_variable'] # _target = 'medv' # default Target cclass for Boston data
    _target_class = config['target_variable'] # _target_class = 'medv'
    print("Input data : {}".format(config['input_file']))
_pandas_profiling = config['profile'] # _pandas_profiling = True # Default is True which gives detailed 

# Silent preprocessing should be True
_silent_preprocessing = True # silent_preproccessing = True

# Temp args
if config['user_selected_model'] == 'Auto select best model':
    _autoselect = True
else:
    autoselect = False
    
_kfold = 10 # config['Kfolds'] # 10
_round = 4
_sort_best_model_by = config['auto_select_best_model_by'] # 'R2' #['R2']
_turbo = True
_blacklist = None
_user_selected_model = None
_unseen_predictions = False
# Now create model
number_of_models = 1
_auto_tune = config['auto_tune'] 


In [2]:
from pycaret.datasets import get_data
dataset = get_data('credit')

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,90000,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
2,50000,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
3,50000,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
4,50000,1,1,2,37,0,0,0,0,0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0


In [3]:
#check the shape of data
dataset.shape

(24000, 24)

In [4]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (22800, 24)
Unseen Data For Predictions: (1200, 24)


In [5]:
from pycaret.classification import *

In [7]:
exp_clf101 = setup(data = data, target = 'default', session_id=123) 

 
Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(22800, 24)"
4,Missing Values,False
5,Numeric Features,14
6,Categorical Features,9
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [8]:

models_comparison = compare_models(blacklist = _blacklist, fold = config['Kfolds'],  round = _round,  sort = config['auto_select_best_model_by'], turbo = _turbo)
models_comparison.to_excel('{}/tmp_models.xlsx'.format(output_folder))
models_comparison_df = pd.read_excel('{}/tmp_models.xlsx'.format(output_folder),index = False)
display(models_comparison)
best_model = models_comparison_df['Model'][0]
top_five_best_models = models_comparison_df['Model'][0:5]
top_three_best_models = models_comparison_df['Model'][0:3]
print("Best suggested model based on best value for {} is {}".format(best_model, _sort_best_model_by))
if _user_selected_model is not None:
    print('Model of interest is selected {}'.format(_user_selected_model))

if _autoselect == True or _user_selected_model is None:
    model_selected = best_model
    print("Selected {} based on {}".format(best_model, _sort_best_model_by))
else:
    model_selected = _user_selected_model
    print("Selected {} based on user choice".format(model_selected))


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Ridge Classifier,0.8236,0,0.3646,0.6932,0.4776,0.3836
1,Linear Discriminant Analysis,0.8236,0.7703,0.3813,0.6818,0.4888,0.3923
2,Gradient Boosting Classifier,0.8225,0.7887,0.3649,0.687,0.4763,0.3813
3,Extreme Gradient Boosting,0.8218,0.7894,0.3595,0.6862,0.4715,0.3767
4,CatBoost Classifier,0.8216,0.7867,0.3864,0.6678,0.4893,0.3904
5,Light Gradient Boosting Machine,0.8214,0.7859,0.3878,0.6663,0.49,0.3908
6,Ada Boost Classifier,0.8185,0.7783,0.3507,0.6729,0.4607,0.3644
7,Extra Trees Classifier,0.8093,0.7533,0.3839,0.61,0.4711,0.362
8,Random Forest Classifier,0.8084,0.738,0.3337,0.6254,0.4349,0.3323
9,Quadratic Discriminant Analysis,0.7893,0.739,0.1734,0.6276,0.2378,0.1698


In [18]:
dt = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7325,0.6213,0.4221,0.4005,0.4110,0.2381
1,0.7237,0.6199,0.4306,0.3878,0.4081,0.2285
2,0.7400,0.6361,0.4504,0.4184,0.4338,0.2654
3,0.7155,0.5984,0.3853,0.3646,0.3747,0.1907
4,0.7375,0.6175,0.4023,0.4057,0.4040,0.2356
5,0.7356,0.6376,0.4618,0.4127,0.4358,0.2639
6,0.7237,0.6210,0.4363,0.3889,0.4112,0.2315
7,0.7387,0.6365,0.4533,0.4167,0.4342,0.2647
8,0.7262,0.6062,0.3909,0.3833,0.3871,0.2108
9,0.7210,0.6030,0.3881,0.3743,0.3811,0.2011


In [19]:
plot_model(tuned_rf, plot = 'auc')

NameError: name 'tuned_rf' is not defined

In [10]:
plot_model(tuned_rf, plot = 'pr')

NameError: name 'tuned_rf' is not defined

In [11]:
plot_model(tuned_rf, plot='feature')

NameError: name 'tuned_rf' is not defined

In [12]:
plot_model(tuned_rf, plot = 'confusion_matrix')

NameError: name 'tuned_rf' is not defined

In [13]:
evaluate_model(tuned_rf)

NameError: name 'tuned_rf' is not defined

In [14]:
predict_model(tuned_rf);

NameError: name 'tuned_rf' is not defined

In [15]:
final_rf = finalize_model(tuned_rf)

NameError: name 'tuned_rf' is not defined

In [16]:
unseen_predictions = predict_model(final_rf, data=data_unseen)
unseen_predictions.head()

NameError: name 'final_rf' is not defined

In [17]:
save_model(final_rf,'Final RF Model 08Feb2020')

NameError: name 'final_rf' is not defined